In [5]:
import pandas as pd
import os
import numpy as np
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.vector_ar.vecm import select_coint_rank

import  scipy.signal.signaltools

#workaround found in Stackoverflow, apparently a VAR function dependency within statsmodel was changed/removed, this workaround defines it again:

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered

from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.vecm import VECM
import datetime
import matplotlib
from statsmodels.tsa.stattools import adfuller
%matplotlib notebook



#If VAR matrix has complete rank, the variables are cointegrated and are an I(0), in this case there is no room for non-stationary time series analysis
#If the VAR matrix has null rank, then the variables are not cointegrated.
#If the VAR matrix has a rank in between, there is cointegration between 2 or more variables.

##The Johansen test is based on the theorem of nullity + rank = rows
##The geometric mutiplicity of Lambda = 0 as an eigenvalue is equal to the NULLITY
##The nullspace of T is the solution to the homogeneus eq.: Ax = 0 (Av = Lambda*v when Lambda = 0)
##The nullity of T is the dimension of the Nullspace, and it is set by the number of free variables in the solution of the equation above (the Nullspace)
##Since the number of free variables mentioned in the previous point (nullity) is set by the number of variables (columns) - the number of equations 
#and the number of equations is equal to the independent rows (rank), then the nullity is equal to the number of columns - the rank.
##For cointegration you will always have a square Matrix, so the nullity is going to be determined only by the number of independent rows (and not by the difference cloumns - rows as well)
##The geometric multiplicity of an eigenvalue is the dimension of the vectors associated with it, in the case of Lambda = 0 it is the nullity
#In the case of the other vectors you just have to create a subspace with all the eigenvectors associated with the eigenvalue an see what dimension of the subspace is.
#In general, all the gemoetric multiplicities are gotten by the dimension of the nullity of A-lambda*I for every lambda
#(for lambda = 0 it turns to the be the nullspace of A because A-0*I = A).
##The algebraic  multiplicity is simply the number of times a certain lambda value appears as a root of the characteristic polynomic of A
#The geometric multiplicity can never exeed the algebraic multiplicity.
#If you add-up all the algebraic multiplicities of every lambda in A (including the nullity of A [geom. multiplicity of lambda = 0]) then you get the number of columns of A. 

##The fundament of the Johansen test is that in Algebra, every nullspace vector corresponds to a linear relationship amongst the variables
##The number of these linear relationships is given by the size of the nullspace.
##This contrast first orders the lambdas from highest to lowest, then (presumably assuming the same distribution for every lambda)
#it does a significance contast with the null hypothesis that the highest lamda is equal to 0, if the hypothesis is rejected, then
#the significance test is repeated with the next lambda. If it is accepted, we assume the rank of the matrix is 0 (if the highest lambda is 0
#we assume the lower lambdas [the subsequent ones in the ordered succesion] will be 0 too.)
##The normalized eigenvectors in relation to the first column give you the cointegration relationships.
##There is a theorem that states that the sum of all the eigenvalues is equal to the trace of the Matrix, that is why there is also a Trace contrast
#and it is because the size of the trace will give you an idea of how big the sum of the eigenvalues is and hence how likely it is that they are far from 0.
##The maximum eigenvalue test examines whether the largest eigenvalue is zero relative to the alternative that the next largest eigenvalue is zero.
##The Johansen test tests for the size of the algebraic multiplicty and not for the size of the geometric multiplicity (the former can be actually higher than the geometric multiplicity)
#So it could understate the number of linearly independent vectors in the VAR matrix (exprssed in error correction form), and hence understate the number of cointegration
#relationships.
#The possible cointegration relations are the eigenvectors becuase they are linear transformations of t-1 variables into t which direction does not change, and that means that they are the possible long term relationships


os.getcwd()

data = pd.read_csv('C:/Users/D110148/OneDrive - pzem/data/Cointegration_Test.csv', sep = ";", decimal=',')
data.index = [pd.to_datetime(i, infer_datetime_format=True) for i in data['Index']]
data = data.iloc[:,1:]
data.where(data == 'NaN', np.nan)
data.interpolate(inplace=True)

asOfDate = datetime.datetime(2021,9,1)

data_to_coint = data[[data.columns[2],data.columns[4],data.columns[5]]]  #1.Gas 2.CO2 3.Coal

data_to_coint.columns = ['TTF','EUA','API2']

#data_to_coint = data[[data.columns[2],data.columns[4],data.columns[7]]]  #1.Gas 2.CO2 3.Power
results = coint_johansen(data_to_coint[data_to_coint.index > asOfDate], det_order = 0,k_ar_diff=2)

data_to_coint.to_csv('dataToCoint.csv')

In [6]:
#Max eigenvalue contrast

results.max_eig_stat, results.max_eig_stat_crit_vals

(array([16.88671259, 14.25804973,  3.07475242]),
 array([[18.8928, 21.1314, 25.865 ],
        [12.2971, 14.2639, 18.52  ],
        [ 2.7055,  3.8415,  6.6349]]))

In [99]:
#Trace contrast:

results.trace_stat, results.trace_stat_crit_vals

(array([34.21951474, 17.33280215,  3.07475242]),
 array([[27.0669, 29.7961, 35.4628],
        [13.4294, 15.4943, 19.9349],
        [ 2.7055,  3.8415,  6.6349]]))

In [100]:
#Rank according to the contrast:

results2=select_coint_rank(data_to_coint[data_to_coint.index > datetime.datetime(2021,6,23)], det_order = 0,k_ar_diff=2)

results2.rank

1

In [101]:
#Eigenvalues

results.eig

array([0.03201338, 0.02709823, 0.00590686])

In [102]:
#Eigenvectors

results.evec

array([[ 0.02220395,  0.03284604, -0.01023705],
       [ 0.07410655, -0.03441259,  0.0326037 ],
       [-0.01330999, -0.0269979 , -0.00785098]])

In [103]:
#Cointegration relationship in case it exists:

vector = results.evec[:,0]/results.evec[2][0]  #1.Gas 2.CO2 3.Coal
vector

array([-1.6682167 , -5.56773887,  1.        ])

In [115]:
#Check with Linear Regression

from sklearn.linear_model import LinearRegression

dataRegress = data_to_coint[data_to_coint.index > asOfDate]

LR = LinearRegression()

LR.fit(np.array(dataRegress.drop(['EUA'], axis=1)),dataRegress.iloc[:,1])

LR.coef_, LR.intercept_, dataRegress

#from statsmodels.tsa.api import VAR

#model = VAR(dataRegress)

#resultsVAR = model.fit(2)

#resultsVAR.summary()

(array([-0.01774345,  0.04008733]),
 76.29192620250986,
                TTF     EUA    API2
 2021-11-01  16.463  34.855   69.90
 2021-12-01  16.857  35.415   73.20
 2021-09-02  17.400  38.760   67.86
 2021-10-02  17.287  39.610   66.83
 2021-11-02  16.925  39.835   66.78
 ...            ...     ...     ...
 2023-08-25  52.108  86.020  128.30
 2023-08-28  55.080  85.755  128.30
 2023-08-29  52.916  84.940  127.84
 2023-08-30  54.451  85.920  128.21
 2023-08-31  52.422  85.995  125.70
 
 [522 rows x 3 columns])

In [105]:
# Trying random experiment with linear relationiship between process 1 and 2

import random
import matplotlib.pyplot as plt
%matplotlib notebook

random.seed(6)

yields1 = [random.normalvariate(0,0.012) for i in range(365)]

process1 = []

P0 = 15

process1.append(P0)

for i in yields1:
    P0 = P0*(1+i)
    process1.append(P0)
    
process2 = [i*0.35 for i in process1]

yields3 = [random.normalvariate(0,0.014) for i in range(365)]

process3 = []

P0 = 20

process3.append(P0)

for i in yields3:
    P0 = P0*(1+i)
    process3.append(P0)
    
    
randomData = pd.DataFrame({'process_1':process1,'process_2':process2,'process_3':process3})

In [106]:
results4 = select_coint_rank(randomData,det_order=0,k_ar_diff=2)

#The rank of the error correction model matrix is (according to the significance Test):

results4.rank

1

In [107]:
results5 = coint_johansen(randomData, det_order=0, k_ar_diff=2)

#Significance test:

results5.max_eig_stat, results5.max_eig_stat_crit_vals

(array([36.12356219,  4.71305756,  0.8677315 ]),
 array([[18.8928, 21.1314, 25.865 ],
        [12.2971, 14.2639, 18.52  ],
        [ 2.7055,  3.8415,  6.6349]]))

In [108]:
#The eigenvectors are:

results5.evec

array([[ 8.93572358e+02,  3.48978776e+00,  3.95428224e+00],
       [-2.55304849e+03, -1.36999180e+01, -1.14832792e+01],
       [ 1.97616097e-03,  1.56242395e-01, -4.57127043e-01]])

In [109]:
#Since the rank is one, we only take the first eigenvector, because there is only 1 cointegration relationship
#We normalize the eigenvector with regard to the first variable:

vector = results5.evec[:,0]/results5.evec[0][0]
vector

array([ 1.00000000e+00, -2.85712563e+00,  2.21152876e-06])

In [110]:
#So the coefficient we are interested in is -2.85712563e+00, because there is only cointegration between the first two variables
#Let's check that that the inverse of the coefficient is 0.35, as we defined process2 = process1*0.35:

1/vector[1]

-0.3500021102931373

In [111]:
#Now let's check that the coefficient -2.85712563e+00 is also achievable through a linear regression:

from sklearn.linear_model import LinearRegression

LinearReg = LinearRegression()

LinearReg.fit(randomData.iloc[:,1:] , randomData.iloc[:,0])

LinearReg.coef_



array([2.85714286, 0.        ])

In [112]:
# Trying random experiment with linear relationiship between process 1,2 and 3

import random
import matplotlib.pyplot as plt
%matplotlib notebook

random.seed(6)

yields1 = [random.normalvariate(0,0.012) for i in range(365)]

process1 = []

P0 = 15

process1.append(P0)

for i in yields1:
    P0 = P0*(1+i)
    process1.append(P0)
    
process2 = [i*0.35 for i in process1]

process3 = [i*0.70 for i in process1]
    
yields4 = [random.normalvariate(0,0.014) for i in range(365)]

process4 = []

P0 = 20

process4.append(P0)

for i in yields4:
    P0 = P0*(1+i)
    process4.append(P0)
    
    
randomData = pd.DataFrame({'process_1':process1,'process_2':process2,'process_3':process3, 'process_4': process4})

In [113]:
results6 = select_coint_rank(randomData, det_order=0,k_ar_diff=2)

results6.rank

LinAlgError: Singular matrix